# File System Tool

The `uwtools` API's `fs` module provides functions to copy and link files as well as create directories. 

For more information, please see the <a href="https://uwtools.readthedocs.io/en/main/sections/user_guide/api/fs.html">uwtools.api.fs</a> Read the Docs page.

In [1]:
from uwtools.api import fs
from pathlib import Path

## Copying Files

The `copy()` function copies files with specified names and can optionally create subdirectories for the copied files to be stored in.

In [2]:
help(fs.copy)

Help on function copy in module uwtools.api.fs:

copy(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, keys: Optional[list[str]] = None, dry_run: bool = False, stdin_ok: bool = False) -> bool
    Copy files.

    :param config: YAML-file path, or ``dict`` (read ``stdin`` if missing or ``None``).
    :param target_dir: Path to target directory.
    :param cycle: A datetime object to make available for use in the config.
    :param leadtime: A timedelta object to make available for use in the config.
    :param keys: YAML keys leading to file dst/src block.
    :param dry_run: Do not copy files.
    :param stdin_ok: OK to read from ``stdin``?
    :return: ``True`` if all copies were created.



Files to be copied are specified using a configuration YAML file, with source paths and names of the files to copy given as the values and the destination paths and names of copied files given as corresponding keys.

In [3]:
%%bash
cat fixtures/fs/copy-config.yaml

file1-copy.nml: fixtures/fs/file1.nml
data/file2-copy.txt: fixtures/fs/file2.txt
data/file3-copy.csv: fixtures/fs/file3.csv


With these instructions, `copy()` creates a copy of each given file with the given name and in the given subdirectory. Copies are created in the directory indicated by `target_dir`. Paths can be provided either as a string or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object. Any directories in the targeted paths for copying will be created if they don't already exist. `True` is returned upon a successful copy.

In [4]:
fs.copy(
    config="fixtures/fs/copy-config.yaml",
    target_dir=Path("tmp/copy-target")
)

True

Examining the target directory, we can see that the copies of the files have been made with their specified names and in their specified directories.

In [5]:
%%bash
tree tmp/copy-target

tmp/copy-target
├── data
│   ├── file2-copy.txt
│   └── file3-copy.csv
└── file1-copy.nml

2 directories, 3 files


### Failing to copy

A configuration can be provided as a dictionary instead as this example demonstrates. However, `missing-file.nml` does not exist. The function provides a warning and returns `False`.

In [6]:
fs.copy(
    config={"missing-copy.nml":"fixtures/fs/missing-file.nml"},
    target_dir="tmp/copy-target"
)

File fixtures/fs/missing-file.nml: State: Pending (EXTERNAL)
Copy fixtures/fs/missing-file.nml -> tmp/copy-target/missing-copy.nml: Final state: Pending
File copies: Final state: Pending


False

The missing copy does not appear in the targeted directory.

In [7]:
%%bash
tree tmp/copy-target

tmp/copy-target
├── data
│   ├── file2-copy.txt
│   └── file3-copy.csv
└── file1-copy.nml

2 directories, 3 files


### Using the `keys` parameter

If the destination keys are not found at the top level of the configuration file, like in the example below, the keys under which this block exists can be provided to the `keys` parameter. If `keys` is not specified with this config file, `copy()` raises a `UWConfigError`. 

In [8]:
%%bash
cat fixtures/fs/copy-keys-config.yaml

files:
  to:
    copy:
      file1-copy.nml: fixtures/fs/file1.nml
      data/file2-copy.txt: fixtures/fs/file2.txt
      data/file3-copy.csv: fixtures/fs/file3.csv


A list of string names is given to the `keys` parameter, each of which represents a key and are ordered from the top level down. The last key in the list directly contains the block of destination keys and their corresponding source values.

In [9]:
fs.copy(
    config="fixtures/fs/copy-keys-config.yaml",
    target_dir="tmp/copy-keys-target",
    keys=["files","to","copy"]
)

True

With this information provided, the copy is successful.

In [10]:
%%bash
tree tmp/copy-keys-target

tmp/copy-keys-target
├── data
│   ├── file2-copy.txt
│   └── file3-copy.csv
└── file1-copy.nml

2 directories, 3 files


### Using the `Copier` class

An alternative to using `copy()` is using the `Copier` class to stage files to copy and carry out the copy with `Copier.go()`.

In [11]:
help(fs.Copier)

Help on class Copier in module uwtools.fs:

class Copier(FileStager)
 |  Copier(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, keys: Optional[list[str]] = None, dry_run: bool = False) -> None
 |
 |  Stage files by copying.
 |
 |  Method resolution order:
 |      Copier
 |      FileStager
 |      Stager
 |      abc.ABC
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  go = __iotaa_tasks__(*args, **kwargs) -> '_AssetT'
 |      Copy files.
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  __abstractmethods__ = frozenset()
 |
 |  ----------------------------------------------------------------------
 |  Methods inherited from Stager:
 |
 |  __init__(self, config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, No

A `Copier` class is instantiated using the same parameters as `copy()`, but copying is not performed until `Copier.go()` is called.

In [12]:
copier = fs.Copier(
    config="fixtures/fs/copy-config.yaml",
    target_dir="tmp/copier-target"
)
copier.go()

[Asset(ref=PosixPath('tmp/copier-target/file1-copy.nml'), ready=<bound method Path.is_file of PosixPath('tmp/copier-target/file1-copy.nml')>),
 Asset(ref=PosixPath('tmp/copier-target/data/file2-copy.txt'), ready=<bound method Path.is_file of PosixPath('tmp/copier-target/data/file2-copy.txt')>),
 Asset(ref=PosixPath('tmp/copier-target/data/file3-copy.csv'), ready=<bound method Path.is_file of PosixPath('tmp/copier-target/data/file3-copy.csv')>)]

Once `Copier.go()` is called, copies are created in the same way as they would have with `copy()`.

In [13]:
%%bash
tree tmp/copier-target

tmp/copier-target
├── data
│   ├── file2-copy.txt
│   └── file3-copy.csv
└── file1-copy.nml

2 directories, 3 files


## Linking files

The `link()` function creates symbolic links to files with specified names and can optionally create subdirectories for the links to be stored in.

In [14]:
help(fs.link)

Help on function link in module uwtools.api.fs:

link(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, keys: Optional[list[str]] = None, dry_run: bool = False, stdin_ok: bool = False) -> bool
    Link files.

    :param config: YAML-file path, or ``dict`` (read ``stdin`` if missing or ``None``).
    :param target_dir: Path to target directory.
    :param cycle: A datetime object to make available for use in the config.
    :param leadtime: A timedelta object to make available for use in the config.
    :param keys: YAML keys leading to file dst/src block.
    :param dry_run: Do not link files.
    :param stdin_ok: OK to read from ``stdin``?
    :return: ``True`` if all links were created.



The links to be created are specified using a configuration YAML file, with source paths and names of the files to link to given as the values and the destination paths and names of symbolic links given as corresponding keys.

In [15]:
%%bash
cat fixtures/fs/link-config.yaml

file1-link.nml: fixtures/fs/file1.nml
file2-link.txt: fixtures/fs/file2.txt
data/file3-link.csv: fixtures/fs/file3.csv


With these instructions, `link()` creates a symbolic link of each given file with the given name and in the given subdirectory. Links are created in the directory indicated by `target_dir`. Paths can be provided either as a string or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object. Any directories in the targeted paths will be created if they don't already exist. `True` is returned upon a successful run.

In [16]:
fs.link(
    config=Path("fixtures/fs/link-config.yaml"),
    target_dir="tmp/link-target"
)

True

Examining the target directory, we can see that the links have been created with their specified names and in their specified directories.

In [17]:
%%bash
tree tmp/link-target

tmp/link-target
├── data
│   └── file3-link.csv -> ../../../fixtures/fs/file3.csv
├── file1-link.nml -> ../../fixtures/fs/file1.nml
└── file2-link.txt -> ../../fixtures/fs/file2.txt

2 directories, 3 files


### Failing to link

A configuration can be provided as a dictionary instead as this example demonstrates. However, `missing-file.nml` does not exist. The function provides a warning and returns `False`.

In [18]:
fs.link(
    config={"missing-link.nml":"fixtures/fs/missing-file.nml"},
    target_dir="tmp/link-target"
)

Filesystem item fixtures/fs/missing-file.nml: State: Pending (EXTERNAL)
Link tmp/link-target/missing-link.nml -> fixtures/fs/missing-file.nml: Final state: Pending
File links: Final state: Pending


False

The missing link does not appear in the targeted directory.

In [19]:
%%bash
tree tmp/link-target

tmp/link-target
├── data
│   └── file3-link.csv -> ../../../fixtures/fs/file3.csv
├── file1-link.nml -> ../../fixtures/fs/file1.nml
└── file2-link.txt -> ../../fixtures/fs/file2.txt

2 directories, 3 files


### Using the `keys` parameter

If the destination keys are not found at the top level of the configuration file, like in the example below, the keys under which this block exists can be provided to the `keys` parameter. If `keys` is not specified with this config file, `link()` raises a `UWConfigError`. 

In [20]:
%%bash
cat fixtures/fs/link-keys-config.yaml

files:
  to:
    link:
      file1-link.nml: fixtures/fs/file1.nml
      file2-link.txt: fixtures/fs/file2.txt
      data/file3-link.csv: fixtures/fs/file3.csv


A list of string names is given to the `keys` parameter, each of which represents a key and are ordered from the top level down. The last key in the list directly contains the block of destination keys and their corresponding source values.

In [21]:
fs.link(
    config="fixtures/fs/link-keys-config.yaml",
    target_dir="tmp/link-keys-target",
    keys=["files","to","link"]
)

True

With this information provided, the links are successfully created.

In [22]:
%%bash
tree tmp/link-keys-target

tmp/link-keys-target
├── data
│   └── file3-link.csv -> ../../../fixtures/fs/file3.csv
├── file1-link.nml -> ../../fixtures/fs/file1.nml
└── file2-link.txt -> ../../fixtures/fs/file2.txt

2 directories, 3 files


### Using the `Linker` class

An alternative to using `link()` is using the `Linker` class to stage files to link and carry out the action with `Linker.go()`.

In [23]:
help(fs.Linker)

Help on class Linker in module uwtools.fs:

class Linker(FileStager)
 |  Linker(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, keys: Optional[list[str]] = None, dry_run: bool = False) -> None
 |
 |  Stage files by linking.
 |
 |  Method resolution order:
 |      Linker
 |      FileStager
 |      Stager
 |      abc.ABC
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  go = __iotaa_tasks__(*args, **kwargs) -> '_AssetT'
 |      Link files.
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  __abstractmethods__ = frozenset()
 |
 |  ----------------------------------------------------------------------
 |  Methods inherited from Stager:
 |
 |  __init__(self, config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, No

A `Linker` class is instantiated using the same parameters as `link()`, but links are not created until `Linker.go()` is called.

In [24]:
linker = fs.Linker(
    config="fixtures/fs/link-config.yaml",
    target_dir="tmp/linker-target"
)
linker.go()

[Asset(ref=PosixPath('tmp/linker-target/file1-link.nml'), ready=<bound method Path.exists of PosixPath('tmp/linker-target/file1-link.nml')>),
 Asset(ref=PosixPath('tmp/linker-target/file2-link.txt'), ready=<bound method Path.exists of PosixPath('tmp/linker-target/file2-link.txt')>),
 Asset(ref=PosixPath('tmp/linker-target/data/file3-link.csv'), ready=<bound method Path.exists of PosixPath('tmp/linker-target/data/file3-link.csv')>)]

Once `Linker.go()` is called, links are created in the same way as they would have with `link()`.

In [25]:
%%bash
tree tmp/linker-target

tmp/linker-target
├── data
│   └── file3-link.csv -> ../../../fixtures/fs/file3.csv
├── file1-link.nml -> ../../fixtures/fs/file1.nml
└── file2-link.txt -> ../../fixtures/fs/file2.txt

2 directories, 3 files


## Creating directories

The `makedirs()` function creates directories.

In [26]:
help(fs.makedirs)

Help on function makedirs in module uwtools.api.fs:

makedirs(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, keys: Optional[list[str]] = None, dry_run: bool = False, stdin_ok: bool = False) -> bool
    Make directories.

    :param config: YAML-file path, or ``dict`` (read ``stdin`` if missing or ``None``).
    :param target_dir: Path to target directory.
    :param cycle: A datetime object to make available for use in the config.
    :param leadtime: A timedelta object to make available for use in the config.
    :param keys: YAML keys leading to file dst/src block.
    :param dry_run: Do not link files.
    :param stdin_ok: OK to read from ``stdin``?
    :return: ``True`` if all directories were made.



Directories to be created are specified using a configuration YAML file, with a required `makedirs` key containing a list of directories and subdirectories to create.

In [27]:
%%bash
cat fixtures/fs/dir-config.yaml

makedirs:
  - foo
  - bar/baz


With these instructions, `makedirs()` creates each directory in the list within the directory indicated by `target_dir`. Paths can be provided either as a string or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object. `True` is returned upon a successful run.

In [28]:
fs.makedirs(
    config="fixtures/fs/dir-config.yaml",
    target_dir=Path("tmp/dir-target")
)

True

Examining the target directory, we can see that the directories have been created with their specified names.

In [29]:
%%bash
tree tmp/dir-target

tmp/dir-target
├── bar
│   └── baz
└── foo

4 directories, 0 files


### Using the `keys` parameter

If the `makedirs` key is not found at the top level of the configuration file, like in the example below, the keys under which this block exists can be provided to the `keys` parameter. If `keys` is not specified with this config file, `makedirs()` raises a `UWConfigError`. 

In [30]:
%%bash
cat fixtures/fs/dir-keys-config.yaml

path:
  to:
    dirs:
      makedirs:
        - foo/bar
        - baz

A list of string names is given to the `keys` parameter, each of which represents a key and are ordered from the top level down. The last key in the list directly contains the block of destination keys and their corresponding source values.

In [31]:
fs.makedirs(
    config="fixtures/fs/dir-keys-config.yaml",
    target_dir="tmp/dir-keys-target",
    keys=["path","to","dirs"]
)

True

With this information provided, the directories are successfully created.

In [32]:
%%bash
tree tmp/dir-keys-target

tmp/dir-keys-target
├── baz
└── foo
    └── bar

4 directories, 0 files


### Using the `MakeDirs` class

An alternative to using `makedirs()` is using the `MakeDirs` class to stage directories to create and carry out the action with `MakeDirs.go()`.

In [33]:
help(fs.MakeDirs)

Help on class MakeDirs in module uwtools.fs:

class MakeDirs(Stager)
 |  MakeDirs(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, keys: Optional[list[str]] = None, dry_run: bool = False) -> None
 |
 |  Make directories.
 |
 |  Method resolution order:
 |      MakeDirs
 |      Stager
 |      abc.ABC
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  go = __iotaa_tasks__(*args, **kwargs) -> '_AssetT'
 |      Make directories.
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  __abstractmethods__ = frozenset()
 |
 |  ----------------------------------------------------------------------
 |  Methods inherited from Stager:
 |
 |  __init__(self, config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None,

A `MakeDirs` class is instantiated using the same parameters as `makedirs()`, but directories are not created until `MakeDirs.go()` is called.

In [34]:
dirs_stager = fs.MakeDirs(
    config="fixtures/fs/dir-config.yaml",
    target_dir="tmp/makedirs-target"
)
dirs_stager.go()

[Asset(ref=PosixPath('tmp/makedirs-target/foo'), ready=<bound method Path.is_dir of PosixPath('tmp/makedirs-target/foo')>),
 Asset(ref=PosixPath('tmp/makedirs-target/bar/baz'), ready=<bound method Path.is_dir of PosixPath('tmp/makedirs-target/bar/baz')>)]

Once `MakeDirs.go()` is called, directories are created in the same way as they would have with `makedirs()`.

In [35]:
%%bash
tree tmp/makedirs-target

tmp/makedirs-target
├── bar
│   └── baz
└── foo

4 directories, 0 files
